Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
 # labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset#, labels
#train_dataset, train_labels = reformat(train_dataset, train_labels)
#valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
#test_dataset, test_labels = reformat(test_dataset, test_labels)
train_dataset= reformat(train_dataset)
valid_dataset = reformat(valid_dataset)
test_dataset = reformat(test_dataset)


print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000,)
Validation set (10000, 784) (10000,)
Test set (10000, 784) (10000,)


In [ ]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [ ]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases 
  l2_loss =tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+ 0.01 * l2_loss
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [ ]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    #print(predictions.shape)
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

In [ ]:
batch_size = 128
hidden_node_size = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights ={ 
      'w': tf.Variable(tf.truncated_normal([image_size * image_size, hidden_node_size])),
      'out': tf.Variable(tf.truncated_normal([hidden_node_size, num_labels]))
  }
  biases = {
      'b':tf.Variable(tf.zeros([hidden_node_size])),
      'out':tf.Variable(tf.zeros([num_labels])) 
  }
  def one_layer_relu(x,weights,biases):
      layer = tf.matmul(x, weights['w']) + biases['b']
      layer = tf.nn.relu(layer)
      out_layer = tf.matmul(layer, weights['out']) + biases['out']
      return out_layer

  pred = one_layer_relu(tf_train_dataset,weights,biases)
  pred_valid = one_layer_relu(tf_valid_dataset,weights,biases)
  pred_test = one_layer_relu(tf_test_dataset,weights,biases)
  l2_loss_relu = tf.nn.l2_loss(weights['w']) +tf.nn.l2_loss(weights['out'])
  # Training computation.
  #logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=pred)) #+0.01 * l2_loss_relu
   
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(pred)
  valid_prediction = tf.nn.softmax(pred_valid)
  test_prediction = tf.nn.softmax(pred_test)

In [ ]:
num_steps = 3000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [ ]:
batch_size = 128
hidden_node_size = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights ={ 
      'w': tf.Variable(tf.truncated_normal([image_size * image_size, hidden_node_size])),
      'out': tf.Variable(tf.truncated_normal([hidden_node_size, num_labels]))
  }
  biases = {
      'b':tf.Variable(tf.zeros([hidden_node_size])),
      'out':tf.Variable(tf.zeros([num_labels])) 
  }
  def one_layer_relu(x,weights,biases):
      layer = tf.matmul(x, weights['w']) + biases['b']
      layer = tf.nn.relu(layer)
      out_layer = tf.matmul(layer, weights['out']) + biases['out']
      return out_layer

  pred = one_layer_relu(tf_train_dataset,weights,biases)
  pred_valid = one_layer_relu(tf_valid_dataset,weights,biases)
  pred_test = one_layer_relu(tf_test_dataset,weights,biases)
  l2_loss_relu = tf.nn.l2_loss(weights['w']) +tf.nn.l2_loss(weights['out'])
  # Training computation.
  #logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=pred))# +0.01 * l2_loss_relu
   
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(pred)
  valid_prediction = tf.nn.softmax(pred_valid)
  test_prediction = tf.nn.softmax(pred_test)

In [ ]:
num_steps = 3000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    train_labels_overfitting =train_labels[:256,:]
    offset = (step * batch_size) % (train_labels_overfitting.shape[0] - batch_size)
    # Generate a minibatch.
    train_dataset_overfitting = train_dataset[:256,:]
    batch_data = train_dataset_overfitting[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [ ]:
batch_size = 128
hidden_node_size = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights ={ 
      'w': tf.Variable(tf.truncated_normal([image_size * image_size, hidden_node_size])),
      'out': tf.Variable(tf.truncated_normal([hidden_node_size, num_labels]))
  }
  biases = {
      'b':tf.Variable(tf.zeros([hidden_node_size])),
      'out':tf.Variable(tf.zeros([num_labels])) 
  }
  def one_layer_relu(x,weights,biases):
      layer = tf.matmul(x, weights['w']) + biases['b']
      layer = tf.nn.dropout(layer,0.6)
      layer = tf.nn.relu(layer)
      out_layer = tf.matmul(layer, weights['out']) + biases['out']
      out_layer = tf.nn.dropout(out_layer, keep_prob=0.6)
      return out_layer
  
  def one_layer_relu_valid_test(x,weights,biases):
      layer = tf.matmul(x, weights['w']) + biases['b']
      #layer = tf.nn.dropout(layer,0.8)
      layer = tf.nn.relu(layer)
      out_layer = tf.matmul(layer, weights['out']) + biases['out']
      #out_layer = tf.nn.dropout(out_layer, keep_prob=0.8)
      return out_layer
    
    
    
    
  pred = one_layer_relu(tf_train_dataset,weights,biases)
  pred_valid = one_layer_relu_valid_test(tf_valid_dataset,weights,biases)
  pred_test = one_layer_relu_valid_test(tf_test_dataset,weights,biases)
  l2_loss_relu = tf.nn.l2_loss(weights['w']) +tf.nn.l2_loss(weights['out'])
  # Training computation.
  #logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=pred)) #+0.01 * l2_loss_relu
   
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(pred)
  valid_prediction = tf.nn.softmax(pred_valid)
  test_prediction = tf.nn.softmax(pred_test)

In [ ]:
num_steps = 3000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    train_labels_overfitting =train_labels[:129,:]
    offset = (step * batch_size) % (train_labels_overfitting.shape[0] - batch_size)
    # Generate a minibatch.
    train_dataset_overfitting = train_dataset[:129,:]
    batch_data = train_dataset_overfitting[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the plceholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [ ]:
batch_size = 128
hidden_node_size = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights ={ 
      'w': tf.Variable(tf.truncated_normal([image_size * image_size, hidden_node_size])),
      'out': tf.Variable(tf.truncated_normal([hidden_node_size, num_labels]))
  }
  biases = {
      'b':tf.Variable(tf.zeros([hidden_node_size])),
      'out':tf.Variable(tf.zeros([num_labels])) 
  }
  def one_layer_relu(x,weights,biases):
      layer = tf.matmul(x, weights['w']) + biases['b']
     # layer = tf.nn.dropout(layer,0.6)
      layer = tf.nn.relu(layer)
      out_layer = tf.matmul(layer, weights['out']) + biases['out']
     # out_layer = tf.nn.dropout(out_layer, keep_prob=0.6)
      return out_layer
  
  def one_layer_relu_valid_test(x,weights,biases):
      layer = tf.matmul(x, weights['w']) + biases['b']
      #layer = tf.nn.dropout(layer,0.8)
      layer = tf.nn.relu(layer)
      out_layer = tf.matmul(layer, weights['out']) + biases['out']
      #out_layer = tf.nn.dropout(out_layer, keep_prob=0.8)
      return out_layer
    
    
    
    
  pred = one_layer_relu(tf_train_dataset,weights,biases)
  pred_valid = one_layer_relu_valid_test(tf_valid_dataset,weights,biases)
  pred_test = one_layer_relu_valid_test(tf_test_dataset,weights,biases)
  l2_loss_relu = tf.nn.l2_loss(weights['w']) +tf.nn.l2_loss(weights['out'])
  # Training computation.
  #logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=pred)) +0.01 * l2_loss_relu
   
    
  #learn_decay
  global_step = tf.Variable(5000) 
  learning_rate = tf.train.exponential_decay(0.5, global_step,10000, 0.96)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(pred)
  valid_prediction = tf.nn.softmax(pred_valid)
  test_prediction = tf.nn.softmax(pred_test)

In [ ]:
num_steps = 5000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #train_labels_overfitting =train_labels[:129,:]
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    #train_dataset_overfitting = train_dataset[:129,:]
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the plceholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

My best straight-forward CNN:
C5x4-C19x8-C5x16-P2-C7x64-P2-C3x256-P3S-C1x1024-C1x512-F2048-F64-F10
where
C5x4 = convolution with 5x5 kernel and 4 maps output
P3S = pooling with 3x3 size of type SAME

ReLU
initial weight SD: 0.05 for conv layers, Xavier for full layers
max pooling
full layer dropout 0,6
conv layer dropout 0,1
conv layer dropout before pooling
shuffle train dataset after each epoch
momentum optimizer with learning rate 0.05
batch size 2048

after 470 epochs (early stop):

train accuracy 99.6%
validation accuracy 94.6%
test accuracy 98.2%

In [15]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
  #print(input_layer.shape)
  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=4,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  conv2 = tf.layers.conv2d(
      inputs=conv1,
      filters=8,
      kernel_size=[19, 19],
      padding="same",
      activation=tf.nn.relu)
 
  conv3 = tf.layers.conv2d(
      inputs=conv2,
      filters=16,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    
  pool1 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    
  conv4 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[7, 7],
      padding="same",
      activation=tf.nn.relu)

  pool2 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)
    
  conv5 = tf.layers.conv2d(
      inputs=pool2,
      filters=256,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  
  pool3 = tf.layers.max_pooling2d(inputs=conv5, pool_size=[3, 3], strides=2)
        
  conv6 = tf.layers.conv2d(
      inputs=pool3,
      filters=1024,
      kernel_size=[1, 1],
      padding="same",
      activation=tf.nn.relu)
    
  conv7 = tf.layers.conv2d(
      inputs=conv6,
      filters=512,
      kernel_size=[1, 1],
      padding="same",
      activation=tf.nn.relu)
    
  flat = tf.reshape(conv7, [-1, 4 * 4 * 512])
  #print(pool2_flat.shape)
  dense1 = tf.layers.dense(inputs=flat, units=2048, activation=tf.nn.relu)
  dropout1 = tf.layers.dropout(
      inputs=dense1, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)
  dense2 = tf.layers.dense(inputs=dropout1, units=64, activation=tf.nn.relu)
  dropout2 = tf.layers.dropout(
      inputs=dense2, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)  
  logits = tf.layers.dense(inputs=dropout2, units=10)
    


  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, 
      logits=logits
  )

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.MomentumOptimizer(
        learning_rate=0.05, 
        momentum=0.9
    )
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [16]:
not_mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="not_mnist_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'not_mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E502202C88>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [18]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_dataset},
    y= train_labels,
    batch_size=2048,
    num_epochs=5,
    shuffle=True)
not_mnist_classifier.train(
    input_fn=train_input_fn,
    hooks=[logging_hook])



INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into not_mnist_convnet_model\model.ckpt.
INFO:tensorflow:probabilities = [[ 0.0984146   0.09861773  0.10243459 ...,  0.0990648   0.10184553
   0.0988103 ]
 [ 0.10046166  0.09809446  0.1005688  ...,  0.10036014  0.10281482
   0.09800126]
 [ 0.09621638  0.10103255  0.10300461 ...,  0.09868801  0.0992009
   0.09880599]
 ..., 
 [ 0.09993975  0.098846    0.10234841 ...,  0.09523343  0.09885948
   0.09608729]
 [ 0.09707577  0.10471201  0.10105933 ...,  0.09900889  0.09888685
   0.09510933]
 [ 0.09854103  0.09976765  0.10038316 ...,  0.09835682  0.10075097
   0.10042205]]
INFO:tensorflow:loss = 2.3034, step = 1


KeyboardInterrupt: 

In [31]:


eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": valid_dataset},
    y=valid_labels,
    num_epochs=1,
    shuffle=False)
eval_results = not_mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2018-01-17-12:59:55
INFO:tensorflow:Restoring parameters from not_mnist_convnet_model\model.ckpt-493
INFO:tensorflow:Finished evaluation at 2018-01-17-12:59:56
INFO:tensorflow:Saving dict for global step 493: accuracy = 0.9535, global_step = 493, loss = 0.160297
{'accuracy': 0.89560002, 'loss': 0.34712443, 'global_step': 492}
INFO:tensorflow:Starting evaluation at 2018-01-17-12:59:56
INFO:tensorflow:Restoring parameters from not_mnist_convnet_model\model.ckpt-493
INFO:tensorflow:Finished evaluation at 2018-01-17-12:59:58
INFO:tensorflow:Saving dict for global step 493: accuracy = 0.8954, global_step = 493, loss = 0.344866
{'accuracy': 0.89539999, 'loss': 0.34486637, 'global_step': 493}


In [32]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_dataset},
    y=test_labels,
    num_epochs=2,
    shuffle=False)
test_results = not_mnist_classifier.evaluate(input_fn=test_input_fn)
print(test_results)

INFO:tensorflow:Starting evaluation at 2018-01-17-13:01:33
INFO:tensorflow:Restoring parameters from not_mnist_convnet_model\model.ckpt-493
INFO:tensorflow:Finished evaluation at 2018-01-17-13:01:34
INFO:tensorflow:Saving dict for global step 493: accuracy = 0.9535, global_step = 493, loss = 0.160297
{'accuracy': 0.95349997, 'loss': 0.16029717, 'global_step': 493}


In [ ]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression

# Data loading and preprocessing
#import tflearn.datasets.oxflower17 as oxflower17
#X, Y = oxflower17.load_data(one_hot=True)
X, Y = tf_train_dataset,tf_train_labels
# Building 'VGG Network'
network = input_data(shape=[None, 28, 28, 1])

network = conv_2d(network, 4, 5, activation='relu')
network = conv_2d(network, 8, 19, activation='relu')
#network = max_pool_2d(network, 2, strides=2)

network = conv_2d(network, 16, 5, activation='relu')
#network = conv_2d(network, 128, 3, activation='relu')
network = max_pool_2d(network, 2, strides=2)

network = conv_2d(network, 64, 7, activation='relu')
#network = conv_2d(network, 256, 3, activation='relu')
#network = conv_2d(network, 256, 3, activation='relu')
network = max_pool_2d(network, 2, strides=2)

network = conv_2d(network, 256, 3, activation='relu')
network = max_pool_2d(network, 3, strides=2)
network = conv_2d(network, 1024, 1, activation='relu')
network = conv_2d(network, 512, 1, activation='relu')
network = fully_connected(network, 2048, activation='relu')
network = dropout(network, 0.6)
network = fully_connected(network, 64, activation='relu')
network = dropout(network, 0.6)
network = fully_connected(network, 10, activation='relu')

#network = conv_2d(network, 512, 3, activation='relu')
#network = conv_2d(network, 512, 3, activation='relu')
#network = max_pool_2d(network, 2, strides=2)

#network = conv_2d(network, 512, 3, activation='relu')
#network = conv_2d(network, 512, 3, activation='relu')
#network = conv_2d(network, 512, 3, activation='relu')
#network = max_pool_2d(network, 2, strides=2)

#network = fully_connected(network, 4096, activation='relu')
#network = dropout(network, 0.5)
#network = fully_connected(network, 4096, activation='relu')
#network = dropout(network, 0.5)
#network = fully_connected(network, 17, activation='softmax')

network = regression(network, optimizer='momentum',
                     loss='categorical_crossentropy',
                     learning_rate=0.05)

# Training
model = tflearn.DNN(network, checkpoint_path='model_vgg',
                    max_checkpoints=1, tensorboard_verbose=0)
model.fit(X, Y, n_epoch=500, shuffle=True,
          show_metric=True, batch_size=2048, #snapshot_step=500,
          snapshot_epoch=False, run_id='test')



In [ ]:
print(train_dataset.reshape(-1,28,28,1).shape)

In [ ]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

In [ ]:
print(train_data.shape)

In [11]:
print(train_dataset[1][1])

-0.5
